# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "recognize"
delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

query = '[lemma="recognize"]'

result = topic_concordancer.main(query,window=20,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [13]:
lemmas = ["recognize","other"]

In [14]:
query = create_contextual_query(lemmas,context_length=3)
print (query)

([lemma="recognize"][]{0,3}[lemma="other"])|([lemma="other"][]{0,3}[lemma="recognize"])


In [15]:
domain_term = "could not recognize each other"

In [16]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [17]:
fragment_1 = {}
fragment_1['original_sentence'] = "Nobody recognized each other. You don't recognize your mother. Your mother didn't recognize you,"
fragment_1['label']="Nobody recognized each other. You don't recognize your mother. Your mother didn't recognize you (..)."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Nobody%22%5D+%5B%22recognized%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22your%22%5D+%5B%22mother%22%5D+%5B%5D+%5B%22Your%22%5D+%5B%22mother%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22you%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "Nobody recognized each other . You do n't recognize your mother . Your mother did n't recognize you , ", 'right': '', 'complete_match': "Nobody recognized each other . You do n't recognize your mother . Your mother did n't recognize you , ", 'testimony_id': 'irn504592', 'shelfmark': ['USHMM RG-50.030*0098'], 'token_start': 3466, 'token_end': 3485}


In [18]:
fragment_2 = {}
fragment_2['original_sentence'] = "we didn't recognize each other because we were without-- sisters didn't recognize each other."
fragment_2['label']="(..)we didn't recognize each other because we were without-- sisters didn't recognize each other."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22we%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22without%22%5D+%5B%5D%7B0%2C3%7D+%5B%22sisters%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "we did n't recognize each other because we were without -- sisters did n't recognize each other . ", 'right': '', 'complete_match': "we did n't recognize each other because we were without -- sisters did n't recognize each other . ", 'testimony_id': 'HVT-134', 'shelfmark': ['Fortunoff Archive HVT-134'], 'token_start': 11904, 'token_end': 11922}


In [19]:
fragment_3 = {}
fragment_3['original_sentence'] = "We did not recognize each other. You know, we're just standing there completely naked with bald head."
fragment_3['label']="We did not recognize each other. You know, we're just standing there completely naked with bald head."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22did%22%5D+%5B%22not%22%5D+%5B%22recognize%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22we%22%5D+%5B%5D+%5B%22just%22%5D+%5B%22standing%22%5D+%5B%22there%22%5D+%5B%22completely%22%5D+%5B%22naked%22%5D+%5B%22with%22%5D+%5B%22bald%22%5D+%5B%22head%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "We did not recognize each other . You know , we 're just standing there completely naked with bald head . ", 'right': '', 'complete_match': "We did not recognize each other . You know , we 're just standing there completely naked with bald head . ", 'testimony_id': 'usc_shoah_10162', 'shelfmark': ['USC Shoah Foundation 10162'], 'token_start': 11700, 'token_end': 11721}


In [20]:
fragment_4 = {}
fragment_4['original_sentence'] = "And we said this is an insane asylum. And we hardly recognized each other."
fragment_4['label']= "And we said this is an insane asylum. And we hardly recognized each other."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22said%22%5D+%5B%22this%22%5D+%5B%22is%22%5D+%5B%22an%22%5D+%5B%22insane%22%5D+%5B%22asylum%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22we%22%5D+%5B%22hardly%22%5D+%5B%22recognized%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we said this is an insane asylum . And we hardly recognized each other . ', 'right': '', 'complete_match': 'And we said this is an insane asylum . And we hardly recognized each other . ', 'testimony_id': 'usc_shoah_1530', 'shelfmark': ['USC Shoah Foundation 1530'], 'token_start': 9806, 'token_end': 9822}


In [21]:
fragment_5 = {}
fragment_5['original_sentence'] = "And when we looked at each other, we couldn't recognize each other, even. We were looking so strange."
fragment_5['label']= "And when we looked at each other, we couldn't recognize each other, even. We were looking so strange."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22when%22%5D+%5B%22we%22%5D+%5B%22looked%22%5D+%5B%22at%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22each%22%5D+%5B%22other%22%5D+%5B%5D+%5B%22even%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22were%22%5D+%5B%22looking%22%5D+%5B%22so%22%5D+%5B%22strange%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And when we looked at each other , we could n't recognize each other , even . We were looking so strange . ", 'right': '', 'complete_match': "And when we looked at each other , we could n't recognize each other , even . We were looking so strange . ", 'testimony_id': 'usc_shoah_26983', 'shelfmark': ['USC Shoah Foundation 26983'], 'token_start': 19395, 'token_end': 19418}


In [22]:
add_testimonial_fragments(fragments)

### 2.  

In [23]:
lemmas = ["recognize","fear"]

In [24]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="recognize"][]{0,10}[lemma="fear"])|([lemma="fear"][]{0,10}[lemma="recognize"])


In [25]:
domain_term = "fear"

In [26]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [27]:
fragment_1 = {}
fragment_1['original_sentence'] = "So every moment, every step that you did, you had a certain fear in yourself. There was a fear of being recognized."
fragment_1['label']="So every moment, every step that you did, you had a certain fear in yourself. There was a fear of being recognized."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22So%22%5D+%5B%22every%22%5D+%5B%22moment%22%5D+%5B%5D+%5B%22every%22%5D+%5B%22step%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22had%22%5D+%5B%22a%22%5D+%5B%22certain%22%5D+%5B%22fear%22%5D+%5B%22in%22%5D+%5B%22yourself%22%5D+%5B%5D+%5B%22There%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22fear%22%5D+%5B%22of%22%5D+%5B%22being%22%5D+%5B%22recognized%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'So every moment , every step that you did , you had a certain fear in yourself . There was a fear of being recognized . ', 'right': '', 'complete_match': 'So every moment , every step that you did , you had a certain fear in yourself . There was a fear of being recognized . ', 'testimony_id': 'irn504823', 'shelfmark': ['USHMM RG-50.030*0329'], 'token_start': 3458, 'token_end': 3484}


In [28]:
fragment_2 = {}
fragment_2['original_sentence'] = "the fear of being recognized, always alert, always listening to people who is coming and who is going,"
fragment_2['label']="(..)the fear of being recognized, always alert, always listening to people who is coming and who is going(..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22fear%22%5D+%5B%22of%22%5D+%5B%22being%22%5D+%5B%22recognized%22%5D+%5B%5D+%5B%22always%22%5D+%5B%22alert%22%5D+%5B%5D+%5B%22always%22%5D+%5B%22listening%22%5D+%5B%22to%22%5D+%5B%22people%22%5D+%5B%22who%22%5D+%5B%22is%22%5D+%5B%22coming%22%5D+%5B%22and%22%5D+%5B%22who%22%5D+%5B%22is%22%5D+%5B%22going%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the fear of being recognized , always alert , always listening to people who is coming and who is going , ', 'right': '', 'complete_match': 'the fear of being recognized , always alert , always listening to people who is coming and who is going , ', 'testimony_id': 'usc_shoah_19939', 'shelfmark': ['USC Shoah Foundation 19939'], 'token_start': 19614, 'token_end': 19635}


In [29]:
fragment_3 = {}
fragment_3['original_sentence'] = "I was always in fear that maybe somebody will recognize me."
fragment_3['label']="I was always in fear that maybe somebody will recognize me."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22always%22%5D+%5B%22in%22%5D+%5B%22fear%22%5D+%5B%22that%22%5D+%5B%22maybe%22%5D+%5B%22somebody%22%5D+%5B%22will%22%5D+%5B%22recognize%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was always in fear that maybe somebody will recognize me . ', 'right': '', 'complete_match': 'I was always in fear that maybe somebody will recognize me . ', 'testimony_id': 'usc_shoah_5328', 'shelfmark': ['USC Shoah Foundation 5328'], 'token_start': 12055, 'token_end': 12067}


In [30]:
fragment_4 = {}
fragment_4['original_sentence'] = "there was this fear that-- that I might be recognized "
fragment_4['label']= "(..) there was this fear that-- that I might be recognized (..)."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22there%22%5D+%5B%22was%22%5D+%5B%22this%22%5D+%5B%22fear%22%5D+%5B%22that%22%5D+%5B%5D%7B0%2C3%7D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22might%22%5D+%5B%22be%22%5D+%5B%22recognized%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'there was this fear that -- that I might be recognized ', 'right': '', 'complete_match': 'there was this fear that -- that I might be recognized ', 'testimony_id': 'usc_shoah_5248', 'shelfmark': ['USC Shoah Foundation 5248'], 'token_start': 13718, 'token_end': 13729}


In [31]:
fragment_5 = {}
fragment_5['original_sentence'] = "But I lived with that fear all the time, that this boy would recognize me."
fragment_5['label']= "But I lived with that fear all the time, that this boy would recognize me."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22But%22%5D+%5B%22I%22%5D+%5B%22lived%22%5D+%5B%22with%22%5D+%5B%22that%22%5D+%5B%22fear%22%5D+%5B%22all%22%5D+%5B%22the%22%5D+%5B%22time%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22this%22%5D+%5B%22boy%22%5D+%5B%22would%22%5D+%5B%22recognize%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'But I lived with that fear all the time , that this boy would recognize me . ', 'right': '', 'complete_match': 'But I lived with that fear all the time , that this boy would recognize me . ', 'testimony_id': 'usc_shoah_7094', 'shelfmark': ['USC Shoah Foundation 7094'], 'token_start': 16614, 'token_end': 16631}


In [32]:
add_testimonial_fragments(fragments)

### 3.  

In [33]:
lemmas = ["recognize","father"]

In [34]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="recognize"][]{0,10}[lemma="father"])|([lemma="father"][]{0,10}[lemma="recognize"])


In [35]:
domain_term = "father"

In [36]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [37]:
fragment_1 = {}
fragment_1['original_sentence'] = "And cut off the beard, cut off the peyos, I couldn't recognize my father. His face was so pale."
fragment_1['label']="And cut off the beard, cut off the peyos, I couldn't recognize my father. His face was so pale."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22cut%22%5D+%5B%22off%22%5D+%5B%22the%22%5D+%5B%22beard%22%5D+%5B%5D+%5B%22cut%22%5D+%5B%22off%22%5D+%5B%22the%22%5D+%5B%22peyos%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%5D+%5B%22His%22%5D+%5B%22face%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22pale%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And cut off the beard , cut off the peyos , I could n't recognize my father . His face was so pale . ", 'right': '', 'complete_match': "And cut off the beard , cut off the peyos , I could n't recognize my father . His face was so pale . ", 'testimony_id': 'usc_shoah_6076', 'shelfmark': ['USC Shoah Foundation 6076'], 'token_start': 15523, 'token_end': 15547}


In [38]:
fragment_2 = {}
fragment_2['original_sentence'] = "so he came to me and he grabbed me, and I could barely recognize my father."
fragment_2['label']="(..) so he came to me and he grabbed me, and I could barely recognize my father."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22so%22%5D+%5B%22he%22%5D+%5B%22came%22%5D+%5B%22to%22%5D+%5B%22me%22%5D+%5B%22and%22%5D+%5B%22he%22%5D+%5B%22grabbed%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%22barely%22%5D+%5B%22recognize%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'so he came to me and he grabbed me , and I could barely recognize my father . ', 'right': '', 'complete_match': 'so he came to me and he grabbed me , and I could barely recognize my father . ', 'testimony_id': 'irn42014', 'shelfmark': ['USHMM RG-50.030*0584'], 'token_start': 10092, 'token_end': 10110}


In [39]:
fragment_3 = {}
fragment_3['original_sentence'] = "And when we looked on my father we didn't recognize him. He looked like a strange person"
fragment_3['label']="And when we looked on my father we didn't recognize him. He looked like a strange person (..)."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22when%22%5D+%5B%22we%22%5D+%5B%22looked%22%5D+%5B%22on%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%22we%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22recognize%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22He%22%5D+%5B%22looked%22%5D+%5B%22like%22%5D+%5B%22a%22%5D+%5B%22strange%22%5D+%5B%22person%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And when we looked on my father we did n't recognize him . He looked like a strange person ", 'right': '', 'complete_match': "And when we looked on my father we did n't recognize him . He looked like a strange person ", 'testimony_id': 'irn504693', 'shelfmark': ['USHMM RG-50.030*0199'], 'token_start': 14717, 'token_end': 14736}


In [40]:
fragment_4 = {}
fragment_4['original_sentence'] = "I could see my father and I hardly recognized my father because he looked like, before Auschwitz like on this photo, he looked so young, not white hair or nothing."
fragment_4['label']= "I could see my father and I hardly recognized my father because (..) he looked so young, not white hair or nothing."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22could%22%5D+%5B%22see%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22hardly%22%5D+%5B%22recognized%22%5D+%5B%22my%22%5D+%5B%22father%22%5D+%5B%22because%22%5D+%5B%22he%22%5D+%5B%22looked%22%5D+%5B%22like%22%5D+%5B%5D+%5B%22before%22%5D+%5B%22Auschwitz%22%5D+%5B%22like%22%5D+%5B%22on%22%5D+%5B%22this%22%5D+%5B%22photo%22%5D+%5B%5D+%5B%22he%22%5D+%5B%22looked%22%5D+%5B%22so%22%5D+%5B%22young%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22white%22%5D+%5B%22hair%22%5D+%5B%22or%22%5D+%5B%22nothing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I could see my father and I hardly recognized my father because he looked like , before Auschwitz like on this photo , he looked so young , not white hair or nothing . ', 'right': '', 'complete_match': 'I could see my father and I hardly recognized my father because he looked like , before Auschwi

In [41]:
add_testimonial_fragments(fragments)